<a href="https://colab.research.google.com/github/JuanIgnacioGarciaUCA/pyFMSD/blob/master/AdvanceNLPwithTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comienzo

In [18]:
import tensorflow as tf
import os
import io
import numpy as np
tf.__version__

'2.8.2'

In [19]:
path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",
origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",
                  extract=True)
# Unzip the file into a folder
!unzip $path_to_zip -d data

Archive:  /root/.keras/datasets/smsspamcollection.zip
replace data/SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/readme? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [20]:
# Let's see if we read the data correctly
lines = io.open('data/SMSSpamCollection').read().strip().split('\n')

In [21]:
spam_dataset = []
for line in lines:
  label, text = line.split('\t')
  if label.strip() == 'spam':
    spam_dataset.append((1, text.strip()))
  else:
    spam_dataset.append(((0, text.strip())))
print(spam_dataset[0]) 

(0, 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')


In [22]:
import pandas as pd
df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])
df.describe()

,Spam
count,5574.000000
mean,0.134015
std,0.340699
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [23]:
df.iloc[0]["Message"]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [24]:
import re
def message_length(x):
  # returns total number of characters
  return len(x)
def num_capitals(x):
  _, count = re.subn(r'[A-Z]', '', x) # only works in english
  return count
def num_punctuation(x):
  _, count = re.subn(r'\W', '', x)
  return count

In [25]:
df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df['Message'].apply(message_length)
df.describe()

,Spam,Capitals,Punctuation,Length
count,5574.000000,5574.000000,5574.000000,5574.000000
mean,0.134015,5.621636,18.942591,80.443488
std,0.340699,11.683233,14.825994,59.841746
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


In [26]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,5564,5565,5566,5567,5568,5569,5570,5571,5572,5573
Spam,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,1,1,0,0,0,0
Message,"Go until jurong point, crazy.. Available only ...",Ok lar... Joking wif u oni...,Free entry in 2 a wkly comp to win FA Cup fina...,U dun say so early hor... U c already then say...,"Nah I don't think he goes to usf, he lives aro...",FreeMsg Hey there darling it's been 3 week's n...,Even my brother is not like to speak with me. ...,As per your request 'Melle Melle (Oru Minnamin...,WINNER!! As a valued network customer you have...,Had your mobile 11 months or more? U R entitle...,...,Ok lor... Sony ericsson salesman... I ask shuh...,Ard 6 like dat lor.,Why don't you wait 'til at least wednesday to ...,Huh y lei...,REMINDER FROM O2: To get 2.50 pounds free call...,This is the 2nd time we have tried 2 contact u...,Will ü b going to esplanade fr home?,"Pity, * was in mood for that. So...any other s...",The guy did some bitching but I acted like i'd...,Rofl. Its true to its name
Capitals,3,2,10,2,2,7,2,10,12,14,...,3,1,1,1,14,9,1,2,2,2
Punctuation,28,11,33,16,14,40,17,31,32,30,...,26,5,17,5,30,38,8,16,26,6
Length,111,29,155,49,61,147,77,160,157,154,...,96,19,67,12,147,160,36,57,125,26


In [27]:
len(df[df.Spam==1]),len(df[df.Spam==0]),len(df)

(747, 4827, 5574)

In [28]:
train=df.sample(frac=0.8,random_state=42)
test=df.drop(train.index)
x_train = train[['Length', 'Capitals', 'Punctuation']]
y_train = train[['Spam']]
x_test = test[['Length', 'Capitals', 'Punctuation']]
y_test = test[['Spam']]

In [29]:
# Basic 1-layer neural network model for evaluation
def make_model(input_dims=3, num_units=12):
  model = tf.keras.Sequential()
  # Adds a densely-connected layer with 12 units to the model:
  model.add(tf.keras.layers.Dense(num_units, 
                                  input_dim=input_dims,
                                  activation='relu'))
  # Add a sigmoid layer with a binary output unit:
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', 
                metrics=['accuracy'])
  return model

In [30]:
model = make_model()
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 4s 3ms/step - loss: 1.2216 - accuracy: 0.8233
Epoch 2/10
446/446 [==============================] - 1s 3ms/step - loss: 0.5743 - accuracy: 0.8693
Epoch 3/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3198 - accuracy: 0.9074
Epoch 4/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2546 - accuracy: 0.9166
Epoch 5/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2274 - accuracy: 0.9231
Epoch 6/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2120 - accuracy: 0.9273
Epoch 7/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2039 - accuracy: 0.9305
Epoch 8/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2047 - accuracy: 0.9329
Epoch 9/10
446/446 [==============================] - 1s 3ms/step - loss: 0.1990 - accuracy: 0.9296
Epoch 10/10
446/446 [==============================] - 1s 3ms/step - loss: 0.1976 - accuracy: 0.9327

In [31]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 2ms/step - loss: 0.1997 - accuracy: 0.9283


[0.1996699720621109, 0.9282511472702026]

In [33]:
#y_train_pred = model.predict_classes(x_train)
y_train_pred = (model.predict(x_train) > 0.5).astype("int32")
tf.math.confusion_matrix(tf.constant(y_train.Spam), 
                         y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3790,   77],
       [ 217,  375]], dtype=int32)>

# Continuación
___

[https://stanfordnlp.github.io/stanfordnlp/](https://stanfordnlp.github.io/stanfordnlp/)

[https://stanfordnlp.github.io/stanza/](https://stanfordnlp.github.io/stanza/)

In [2]:
#!pip install stanfordnlp
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import stanza as stnlp
stnlp.download('en')
#stnlp.download('es')

INFO:stanza:Downloading default packages for language: en (English)...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [18]:
#import stanfordnlp as snlp
#snlp.download('en')
#snlp.download('es')

In [40]:
!pip3 install torch --extra-index-url https://download.pytorch.org/whl/cu113
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:14:04tcmalloc: large alloc 1147494400 bytes == 0x398f8000 @  0x7fe3cf821615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:11:24tcmalloc: large alloc 1434370048 bytes == 0x7df4e000 @  0x7fe3cf821615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f6

In [4]:
en_st = stnlp.Pipeline(lang='en') #, processors='tokenize')
es_st = stnlp.Pipeline(lang='es') #, processors='tokenize')

INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [30]:
#en = snlp.Pipeline(lang='en') #, processors='tokenize')
#es = snlp.Pipeline(lang='es') #, processors='tokenize')

In [5]:
sentence = '''Go until Jurong point, crazy.. Available only in bugis
 great world'''

In [7]:
es_st("""Hola cómo estás? Yo estoy bien. 
Creo que ya es tarde.
¿Qué hora es?""")

[
  [
    {
      "id": 1,
      "text": "Hola",
      "lemma": "Hola",
      "upos": "INTJ",
      "head": 2,
      "deprel": "dep",
      "start_char": 0,
      "end_char": 4,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "cómo",
      "lemma": "cómo",
      "upos": "PRON",
      "feats": "PronType=Ind",
      "head": 3,
      "deprel": "obl",
      "start_char": 5,
      "end_char": 9,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "estás",
      "lemma": "estar",
      "upos": "VERB",
      "feats": "Mood=Ind|Number=Sing|Person=2|Tense=Pres|VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "start_char": 10,
      "end_char": 15,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "?",
      "lemma": "?",
      "upos": "PUNCT",
      "feats": "PunctSide=Fin|PunctType=Qest",
      "head": 3,
      "deprel": "punct",


In [8]:
tokenized = en_st(sentence)
len(tokenized.sentences)

2

In [9]:
s0=tokenized.sentences[0]

In [10]:
s0.print_tokens()

<Token id=1;words=[<Word id=1;text=Go;lemma=go;upos=VERB;xpos=VB;feats=Mood=Imp|VerbForm=Fin;head=0;deprel=root>]>
<Token id=2;words=[<Word id=2;text=until;lemma=until;upos=ADP;xpos=IN;head=4;deprel=case>]>
<Token id=3;words=[<Word id=3;text=Jurong;lemma=Jurong;upos=PROPN;xpos=NNP;feats=Number=Sing;head=4;deprel=compound>]>
<Token id=4;words=[<Word id=4;text=point;lemma=point;upos=NOUN;xpos=NN;feats=Number=Sing;head=1;deprel=obl>]>
<Token id=5;words=[<Word id=5;text=,;lemma=,;upos=PUNCT;xpos=,;head=6;deprel=punct>]>
<Token id=6;words=[<Word id=6;text=crazy;lemma=crazy;upos=ADJ;xpos=JJ;feats=Degree=Pos;head=1;deprel=parataxis>]>
<Token id=7;words=[<Word id=7;text=..;lemma=..;upos=PUNCT;xpos=.;head=1;deprel=punct>]>


In [11]:
for snt in tokenized.sentences:
  for word in snt.tokens:
    print(word.text)
  print("<End of Sentence>")

Go
until
Jurong
point
,
crazy
..
<End of Sentence>
Available
only
in
bugis
great
world
<End of Sentence>


In [12]:
s0=tokenized.sentences[0]

In [13]:
s0.print_tokens()

<Token id=1;words=[<Word id=1;text=Go;lemma=go;upos=VERB;xpos=VB;feats=Mood=Imp|VerbForm=Fin;head=0;deprel=root>]>
<Token id=2;words=[<Word id=2;text=until;lemma=until;upos=ADP;xpos=IN;head=4;deprel=case>]>
<Token id=3;words=[<Word id=3;text=Jurong;lemma=Jurong;upos=PROPN;xpos=NNP;feats=Number=Sing;head=4;deprel=compound>]>
<Token id=4;words=[<Word id=4;text=point;lemma=point;upos=NOUN;xpos=NN;feats=Number=Sing;head=1;deprel=obl>]>
<Token id=5;words=[<Word id=5;text=,;lemma=,;upos=PUNCT;xpos=,;head=6;deprel=punct>]>
<Token id=6;words=[<Word id=6;text=crazy;lemma=crazy;upos=ADJ;xpos=JJ;feats=Degree=Pos;head=1;deprel=parataxis>]>
<Token id=7;words=[<Word id=7;text=..;lemma=..;upos=PUNCT;xpos=.;head=1;deprel=punct>]>


In [14]:
tokEs=es_st("Esto es una prueba. Estoy viendo la tele.")
tokEs

[
  [
    {
      "id": 1,
      "text": "Esto",
      "lemma": "este",
      "upos": "PRON",
      "feats": "Number=Sing|PronType=Dem",
      "head": 4,
      "deprel": "nsubj",
      "start_char": 0,
      "end_char": 4,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "es",
      "lemma": "ser",
      "upos": "AUX",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
      "head": 4,
      "deprel": "cop",
      "start_char": 5,
      "end_char": 7,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "una",
      "lemma": "uno",
      "upos": "DET",
      "feats": "Definite=Ind|Gender=Fem|Number=Sing|PronType=Art",
      "head": 4,
      "deprel": "det",
      "start_char": 8,
      "end_char": 11,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "prueba",
      "lemma": "prueba",
      "upos": "NOUN",
      "fea

In [ ]:
tokEs.sentences[1].print_tokens()

<Token id=1;words=[<Word id=1;text=Estoy;lemma=estar;upos=AUX;feats=Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin;head=2;deprel=aux>]>
<Token id=2;words=[<Word id=2;text=viendo;lemma=ver;upos=VERB;feats=VerbForm=Ger;head=0;deprel=root>]>
<Token id=3;words=[<Word id=3;text=la;lemma=el;upos=DET;feats=Definite=Def|Gender=Fem|Number=Sing|PronType=Art;head=4;deprel=det>]>
<Token id=4;words=[<Word id=4;text=tele;lemma=tele;upos=NOUN;feats=Gender=Fem|Number=Sing;head=2;deprel=obj>]>
<Token id=5;words=[<Word id=5;text=.;lemma=.;upos=PUNCT;feats=PunctType=Peri;head=2;deprel=punct>]>


In [15]:
#en = snlp.Pipeline(lang='en')
def word_counts(x, pipeline=en_st):
  doc = pipeline(x)
  count = sum([len(sentence.tokens) for sentence in doc.sentences])
  return count

In [34]:
train['Message'].iloc[5:7].apply(word_counts)

5412    22
4362    24
Name: Message, dtype: int64

In [35]:
len( train['Message'] )

4459

In [36]:
train['Words'] = train['Message'].apply(word_counts)

In [37]:
test['Words'] = test['Message'].apply(word_counts)

In [38]:
x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train[['Spam']]
x_test = test[['Length', 'Punctuation', 'Capitals' , 'Words']]
y_test = test[['Spam']]

In [39]:
model = make_model(input_dims=4)

In [40]:
train.loc[train.Spam == 1].describe()

,Spam,Capitals,Punctuation,Length,Words
count,592.0,592.000000,592.000000,592.000000,592.000000
mean,1.0,15.320946,29.086149,138.856419,28.989865
std,0.0,11.635105,7.083572,28.079980,7.298093
min,1.0,0.000000,2.000000,13.000000,2.000000
25%,1.0,7.000000,26.000000,132.000000,26.000000
50%,1.0,14.000000,30.000000,149.000000,30.000000
75%,1.0,21.000000,34.000000,157.000000,34.000000
max,1.0,128.000000,49.000000,197.000000,46.000000


In [41]:
train.loc[train.Spam == 0].describe()

,Spam,Capitals,Punctuation,Length,Words
count,3867.0,3867.000000,3867.000000,3867.000000,3867.000000
mean,0.0,4.018878,17.325058,71.354538,17.076028
std,0.0,10.599291,14.826644,57.755351,13.599832
min,0.0,0.000000,0.000000,2.000000,1.000000
25%,0.0,1.000000,8.000000,33.000000,8.000000
50%,0.0,2.000000,13.000000,53.000000,13.000000
75%,0.0,3.000000,23.000000,92.000000,22.000000
max,0.0,129.000000,253.000000,910.000000,206.000000


In [42]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 2s 3ms/step - loss: 0.5676 - accuracy: 0.8634
Epoch 2/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3323 - accuracy: 0.9087
Epoch 3/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2792 - accuracy: 0.9195
Epoch 4/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2482 - accuracy: 0.9237
Epoch 5/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2326 - accuracy: 0.9231
Epoch 6/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2203 - accuracy: 0.9296
Epoch 7/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2143 - accuracy: 0.9271
Epoch 8/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2083 - accuracy: 0.9294
Epoch 9/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2025 - accuracy: 0.9289
Epoch 10/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2019 - accuracy: 0.9282

In [43]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 3ms/step - loss: 0.2012 - accuracy: 0.9291


[0.20123417675495148, 0.9291479587554932]

In [44]:
#y_train_pred = model.predict_classes(x_train)
y_test_pred = (model.predict(x_test) > 0.5).astype("int32")
tf.math.confusion_matrix(tf.constant(y_test.Spam), 
                         y_test_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[938,  22],
       [ 57,  98]], dtype=int32)>

https://github.com/stopwords-iso/stopwords-iso

In [45]:
!pip install stopwordsiso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 73 kB 2.2 MB/s 


In [46]:
import stopwordsiso as stopwords
stopwords.langs()
len( sorted(stopwords.stopwords('en')) ),len( sorted(stopwords.stopwords('es')) )

(1298, 732)

In [47]:
en_sw = stopwords.stopwords('en')
def word_counts(x, pipeline=en_st):
  doc = pipeline(x)
  count = 0
  for sentence in doc.sentences:
    for token in sentence.tokens:
        if token.text.lower() not in en_sw:
          count += 1
  return count

In [48]:
train['Words'] = train['Message'].apply(word_counts)

In [49]:
test['Words'] = test['Message'].apply(word_counts)

In [51]:
test.T

,3,4,5,9,16,34,54,55,64,66,...,5519,5520,5522,5527,5539,5548,5553,5554,5562,5564
Spam,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Message,U dun say so early hor... U c already then say...,"Nah I don't think he goes to usf, he lives aro...",FreeMsg Hey there darling it's been 3 week's n...,Had your mobile 11 months or more? U R entitle...,Oh k...i'm watching here:),Thanks for your subscription to Ringtone UK yo...,SMS. ac Sptv: The New Jersey Devils and the De...,Do you know what Mallika Sherawat did yesterda...,Ok lar i double check wif da hair dresser alre...,"Today is ""song dedicated day.."" Which song wil...",...,Miles and smiles r made frm same letters but d...,"By the way, i've put a skip right outside the ...",No. I dont want to hear anything,I want to tell you how bad I feel that basical...,Want explicit SEX in 30 secs? Ring 02073162414...,Ic. There are a lotta childporn cars then.,Wen did you get so spiritual and deep. That's ...,Have a safe trip to Nigeria. Wish you happines...,Anything lor. Juz both of us lor.,Ok lor... Sony ericsson salesman... I ask shuh...
Capitals,2,2,7,14,1,13,24,7,2,3,...,3,2,2,4,17,2,2,3,2,3
Punctuation,16,14,40,30,9,31,26,19,25,30,...,42,30,7,22,18,9,11,17,8,26
Length,49,61,147,154,26,158,120,76,110,122,...,166,122,32,103,90,42,51,91,33,96
Words,4,5,22,13,4,13,20,9,14,16,...,15,7,2,5,16,6,5,8,5,9


In [52]:
x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train[['Spam']]
x_test = test[['Length', 'Punctuation', 'Capitals', 'Words']]
y_test = test[['Spam']]
model = make_model(input_dims=4)
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 2s 3ms/step - loss: 0.8905 - accuracy: 0.7609
Epoch 2/10
446/446 [==============================] - 1s 3ms/step - loss: 0.4650 - accuracy: 0.8569
Epoch 3/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3709 - accuracy: 0.8706
Epoch 4/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3119 - accuracy: 0.8901
Epoch 5/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2732 - accuracy: 0.8995
Epoch 6/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2538 - accuracy: 0.9096
Epoch 7/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2415 - accuracy: 0.9134
Epoch 8/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2285 - accuracy: 0.9170
Epoch 9/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2283 - accuracy: 0.9166
Epoch 10/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2206 - accuracy: 0.9186

In [55]:
model.evaluate(x_train, y_train)

140/140 [==============================] - 0s 2ms/step - loss: 0.2482 - accuracy: 0.9276


[0.2482335865497589, 0.9275622367858887]

In [56]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 4ms/step - loss: 0.2656 - accuracy: 0.9085


[0.2655699551105499, 0.9085201621055603]

In [57]:
#y_train_pred = model.predict_classes(x_train)
y_test_pred = (model.predict(x_test) > 0.5).astype("int32")
tf.math.confusion_matrix(tf.constant(y_test.Spam), 
                         y_test_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[891,  69],
       [ 33, 122]], dtype=int32)>